### Generates Treatment History Confounders

Uses ehealey/omop-learn/Manuscript_ready_files Treatment_History_Confounder_Generation

In [1]:
in_name = 'cohort_manuscript_updated_treatments_ncj'
out_name = 'manuscript_covariates_0_final'

In [2]:
import sys
sys.path.append( '..' )

In [3]:
import psycopg2
import pandas as pd
import numpy
import sparse
import config
import sys
import time
import importlib
#import sparse
import datetime
import pandas as pd
import numpy as np
import scipy.sparse
import matplotlib.pyplot as plt

import Utils.dbutils as dbutils
import Utils.data_utils as data_utils
import Generators.CohortGenerator as CohortGenerator
import Generators.FeatureGenerator as FeatureGenerator
import config
local_imports = (
    dbutils,
    data_utils,
    CohortGenerator,
    FeatureGenerator,
    config
)
for i in local_imports:
    i = importlib.reload(i)

In [4]:
# database connection
username = config.PG_USERNAME
password = config.PG_PASSWORD
database_name = config.DB_NAME

#config_path = 'postgresql://{username}:{password}@{database_name}'.format(
config_path = 'postgresql://{database_name}'.format(database_name = database_name)

connect_args = {"host": '/var/run/postgresql/'} # connect_args to pass to sqlalchemy create_engine function

# schemas 
# schema_name = 'eol_test_ncjones' # all created tables will be created using this schema
cdm_schema_name = config.OMOP_CDM_SCHEMA # the name of the schema housing your OMOP CDM tables
print(f"cdm schema: {cdm_schema_name}")

    #username = username,
    #password = password,
   # database_name = database_name
#)

# schemas 
schema_name = 'eol_test_eheal' # all created tables will be created using this schema

# caching
reset_schema = False # if true, rebuild all data from scratch

# set up database, reset schemas as needed
db = dbutils.Database(config_path, schema_name, connect_args, cdm_schema_name)
# if reset_schema:
#     db.execute(
#         'drop schema if exists {} cascade'.format(schema_name)
#     )
# db.execute(
#     'create schema if not exists {}'.format(schema_name)
# )

cdm schema: cdm_6871_21


In [5]:
%%time
# set input
username='cdm_6871_21'
password='MLHC2021'
database_name = 'omop_v6'
conn_string = "dbname="+database_name + " host=/var/run/postgresql"
conn=psycopg2.connect(conn_string)

sql_cmd = 'select * from cdm_6871_21.{}'.format(in_name)
covariate_df =pd.read_sql(sql_cmd, conn)



CPU times: user 801 ms, sys: 136 ms, total: 937 ms
Wall time: 1.03 s


In [6]:
%%time
# Get all antibiotic history per patient
cohort_treatments = pd.read_sql("""SELECT
                                uti.index,
                                uti.condition_occurrence_id, 
                                uti.person_id,
                                uti.condition_start_date,
                                anti.*,
                                drug.drug_exposure_id,
                                drug.drug_exposure_start_date,
                                drug.drug_exposure_start_datetime
                           FROM cdm_6871_21.cohort_manuscript_updated_treatments_ncj uti
                           JOIN cdm_6871_21.drug_exposure drug ON 
                                    drug.person_id = uti.person_id AND
                                    drug.drug_exposure_start_date - uti.condition_start_date < -1
                           JOIN cdm_6871_21.antibiotics_v2_ncj anti ON
                                       drug.drug_concept_id = anti.drug_concept_id
                           ORDER BY 
                                    uti.condition_occurrence_id
                        """, conn)

CPU times: user 2.3 s, sys: 345 ms, total: 2.65 s
Wall time: 24.6 s


In [9]:
%%time
cohort_v7 = covariate_df.copy()

# Create New Variables 
cohort_v7 = cohort_v7.assign(nitrofurantoin_switch_ever=0)
cohort_v7 = cohort_v7.assign(fosfomycin_switch_ever=0)
cohort_v7['trimethoprim-sulfamethoxazole_switch_ever'] =0
cohort_v7 = cohort_v7.assign(second_line_switch_ever=0)
cohort_v7 = cohort_v7.assign(alternatives_switch_ever=0)
cohort_v7 = cohort_v7.assign(inappropriate_switch_ever=0)

cohort_v7 = cohort_v7.assign(nitrofurantoin_switch_recent=0)
cohort_v7 = cohort_v7.assign(fosfomycin_switch_recent=0)
cohort_v7['trimethoprim-sulfamethoxazole_switch_recent'] =0
cohort_v7 = cohort_v7.assign(second_line_switch_recent=0)
cohort_v7 = cohort_v7.assign(alternatives_switch_recent=0)
cohort_v7 = cohort_v7.assign(inappropriate_switch_recent=0)

cohort_v7 = cohort_v7.assign(nitrofurantoin_1_to_7_days=0)
cohort_v7 = cohort_v7.assign(fosfomycin_1_to_7_days=0)
cohort_v7['trimethoprim-sulfamethoxazole_1_to_7_days'] =0
cohort_v7 = cohort_v7.assign(second_line_1_to_7_days=0)
cohort_v7 = cohort_v7.assign(alternatives_1_to_7_days=0)
cohort_v7 = cohort_v7.assign(inappropriate_1_to_7_days=0)

cohort_v7 = cohort_v7.assign(nitrofurantoin_0_to_6_mo=0)
cohort_v7 = cohort_v7.assign(fosfomycin_0_to_6_mo=0)
cohort_v7['trimethoprim-sulfamethoxazole_0_to_6_mo'] =0
cohort_v7 = cohort_v7.assign(second_line_0_to_6_mo=0)
cohort_v7 = cohort_v7.assign(alternatives_0_to_6_mo=0)
cohort_v7 = cohort_v7.assign(inappropriate_0_to_6_mo=0)

cohort_v7 = cohort_v7.assign(nitrofurantoin_6_to_12_mo=0)
cohort_v7 = cohort_v7.assign(fosfomycin_6_to_12_mo=0)
cohort_v7['trimethoprim-sulfamethoxazole_6_to_12_mo'] =0
cohort_v7 = cohort_v7.assign(second_line_6_to_12_mo=0)
cohort_v7 = cohort_v7.assign(alternatives_6_to_12_mo=0)
cohort_v7 = cohort_v7.assign(inappropriate_6_to_12_mo=0)

cohort_v7 = cohort_v7.assign(nitrofurantoin_12_to_24_mo=0)
cohort_v7 = cohort_v7.assign(fosfomycin_12_to_24_mo=0)
cohort_v7['trimethoprim-sulfamethoxazole_12_to_24_mo'] =0
cohort_v7 = cohort_v7.assign(second_line_12_to_24_mo=0)
cohort_v7 = cohort_v7.assign(alternatives_12_to_24_mo=0)
cohort_v7 = cohort_v7.assign(inappropriate_12_to_24_mo=0)


cohort_v7 = cohort_v7.assign(nitrofurantoin_most_recent=0)
cohort_v7 = cohort_v7.assign(fosfomycin_most_recent=0)
cohort_v7['trimethoprim-sulfamethoxazole_most_recent'] =0
cohort_v7 = cohort_v7.assign(second_line_most_recent=0)
cohort_v7 = cohort_v7.assign(alternatives_most_recent=0)
cohort_v7 = cohort_v7.assign(inappropriate_most_recent=0)

CPU times: user 488 ms, sys: 84.1 ms, total: 572 ms
Wall time: 569 ms


In [10]:
ids_with_previous_treatment = cohort_treatments.condition_occurrence_id.unique()

In [12]:
%%time
i=0
cohort_temp = cohort_v7.copy()
for condition_occurrence_id in ids_with_previous_treatment:
    temp = cohort_treatments[cohort_treatments.condition_occurrence_id==condition_occurrence_id]
    index = list(cohort_temp[cohort_temp.condition_occurrence_id ==condition_occurrence_id].index)[0]
    if temp.antibiotic_type.nunique()>1:
        if 'level_0' in temp.columns:
            temp = temp.drop(columns ='level_0')
        temp = temp.sort_values('drug_exposure_start_date',ascending = False).reset_index()
        for inex, row in temp.iterrows():
            drug_date = row.drug_exposure_start_date
            uti_date = row.condition_start_date
            drug_type = row.antibiotic_type 
            if inex ==0:
                cohort_temp.at[index, str(drug_type)+'_most_recent'] = 1 
                after_drug = drug_type
            else:
                for j in range(inex):
                    row_df =temp.iloc[j,:]
                    drug_date_after = row_df.drug_exposure_start_date 
                    uti_date_after = row_df.condition_start_date
                    drug_type_after = row_df.antibiotic_type 
                    if drug_type_after != drug_type:
                        if (drug_date_after-drug_date).days>0 and (drug_date_after-drug_date).days<8:
                            cohort_temp.at[index, str(drug_type)+'_switch_ever'] = 1
                            if inex == min(temp[temp.antibiotic_type ==drug_type].index):
                                cohort_temp.at[index, str(drug_type)+'_switch_recent'] = 1
            
    elif temp.antibiotic_type.nunique()==1:
        drug_type = list(temp.antibiotic_type)[0]
        cohort_temp.at[index, str(drug_type)+'_most_recent'] = 1 

    for inex, row in temp.iterrows():
        drug_date = row.drug_exposure_start_date
        uti_date = row.condition_start_date
        drug_type = row.antibiotic_type 
        
        if (uti_date-drug_date).days >0 and (uti_date-drug_date).days <8:          
            cohort_temp.at[index, str(drug_type)+'_1_to_7_days'] = 1
        if (uti_date-drug_date).days >0 and (uti_date-drug_date).days <183:          
            cohort_temp.at[index, str(drug_type)+'_0_to_6_mo'] = 1
        if (uti_date-drug_date).days >182 and (uti_date-drug_date).days <365:          
            cohort_temp.at[index, str(drug_type)+'_6_to_12_mo'] = 1        
        if (uti_date-drug_date).days >364 and (uti_date-drug_date).days <728:          
            cohort_temp.at[index, str(drug_type)+'_12_to_24_mo'] = 1    
 
    i = i+1
    if i%1000==0:
        print(str(round(i/len(ids_with_previous_treatment)*100))+'%')

    

2%
4%
6%
8%
11%
13%
15%
17%
19%
21%
23%
25%
27%
29%
32%
34%
36%
38%
40%
42%
44%
46%
48%
51%
53%
55%
57%
59%
61%
63%
65%
67%
70%
72%
74%
76%
78%
80%
82%
84%
86%
88%
91%
93%
95%
97%
99%
CPU times: user 12min 58s, sys: 540 ms, total: 12min 59s
Wall time: 12min 59s


In [15]:
if 'level_0' in cohort_temp.columns:
    cohort_temp =cohort_temp.drop(columns = 'level_0')

In [16]:
cohort_temp.to_sql("{}".format(out_name),con=db.engine, if_exists="replace", schema="cdm_6871_21")

In [17]:
db.execute('GRANT SELECT ON TABLE cdm_6871_21.{} TO cdm_6871_21;'.format(out_name))



Executed 1 SQLs
